In [1]:
import re
import gzip
from tqdm import tqdm
import pandas as pd

from rdkit import Chem
from rdkit.Chem import PandasTools

smiles_to_add = [
    'CO', 'CI', 'CBr', 'CCl', 
    'CCCl', 'CCBr', 'CCI', 
    'CN', 'CCN', 'NC=N', 'NN',
    'C=O', 'CC=O',     
]

### create new stock

In [9]:
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

df = PandasTools.LoadSDF('data/molport/7_Days-1.sdf.gz', molColName='mol').reset_index()
df.info()

,index,Molport ID,SMILES,Supplier Country,Link to Molport,ID,mol
0,0,Molport-047-774-043,C[C@H](OC(=O)OC(C)(C)C)C=C |r|,Latvia,https://www.molport.com/shop/compound/Molport-...,7_Days-1.sdf                                  ...,<rdkit.Chem.rdchem.Mol object at 0x00000273A91...
1,1,Molport-047-774-054,Fc1ccc2c(Br)c[se]c2c1,Latvia,https://www.molport.com/shop/compound/Molport-...,,<rdkit.Chem.rdchem.Mol object at 0x00000272FFD...
2,2,Molport-047-774-041,CON(C)C(=O)[C@H](C)O[Si](c1ccccc1)(c1ccccc1)C(...,Latvia,https://www.molport.com/shop/compound/Molport-...,,<rdkit.Chem.rdchem.Mol object at 0x00000272FFD...
3,3,Molport-047-774-036,O=S(=O)(C1CC1)c1nnnn1-c1ccccc1,Latvia,https://www.molport.com/shop/compound/Molport-...,,<rdkit.Chem.rdchem.Mol object at 0x00000272FFD...
4,4,Molport-047-773-927,ClCC(=O)N(c1ccccc1)c1c(C(=O)CCl)c(=O)oc2ccccc12,Latvia,https://www.molport.com/shop/compound/Molport-...,,<rdkit.Chem.rdchem.Mol object at 0x00000272FFD...
...,...,...,...,...,...,...,...
323087,323279,Molport-021-800-971,Fc1ccc(cc1F)-n1cc(Br)cn1,USA,https://www.molport.com/shop/compound/Molport-...,Chemistry 47149,<rdkit.Chem.rdchem.Mol object at 0x00000274F48...
323088,323280,Molport-006-727-625,COc1ccc2ccnc(Cl)c2c1,USA,https://www.molport.com/shop/compound/Molport-...,,<rdkit.Chem.rdchem.Mol object at 0x00000274F48...
323089,323281,Molport-051-542-081,Cl.NC(C1CC1)c1cccc(Br)c1,USA,https://www.molport.com/shop/compound/Molport-...,,<rdkit.Chem.rdchem.Mol object at 0x00000274F48...
323090,323282,Molport-051-738-701,CC1(C)O[C@@H]2[C@@H](O[C@H](CO)C2(C)O1)n1ccc2c...,USA,https://www.molport.com/shop/compound/Molport-...,,<rdkit.Chem.rdchem.Mol object at 0x00000274F48...


In [ ]:
# pd.Series(Chem.MultithreadedSDMolSupplier('data/molport/3_Days-1.sdf', numWriterThreads=16), name='mol')

In [10]:
for i in tqdm(df.index):
    try:
        # smi = df.at[i, 'SMILES']        
        # mol = Chem.MolFromSmiles(smi)
        
        mol = df.at[i, 'mol']                
        df.at[i, 'inchi_key'] = Chem.MolToInchiKey(mol)
    except Exception as e:        
        pass
        # smi = Chem.MolToSmiles(mol)
        # print(smi, e)


100%|██████████| 323092/323092 [01:49<00:00, 2961.42it/s]


In [11]:
for smi in smiles_to_add:
    mol = Chem.MolFromSmiles(smi)
    df.at[len(df), 'inchi_key'] = Chem.MolToInchiKey(mol)

df.tail()

,index,Molport ID,SMILES,Supplier Country,Link to Molport,ID,mol,inchi_key
323100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,QUSNBJAOOMFDIB-UHFFFAOYSA-N
323101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PNKUSGQVOMIXLU-UHFFFAOYSA-N
323102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OAKJQQAXSVQMHS-UHFFFAOYSA-N
323103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WSFSSNUMVMOOMR-UHFFFAOYSA-N
323104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IKHGUXGNUITLKF-UHFFFAOYSA-N


In [17]:
cols = ['Molport ID', 'SMILES', 'Supplier Country', 'inchi_key']
df = df[cols].dropna(subset='inchi_key')

In [19]:
with pd.HDFStore('config/molport_7d_stock.hdf5', 'w') as store:
    store.put('/table', pd.DataFrame(df.inchi_key.dropna()))

### modify zinc stock

In [4]:
print('Loading zinc stock ... ', end='')
with pd.HDFStore('config/zinc_stock.hdf5', 'r') as store:
    df = store.get('/table')
print('Done')
    
ser = pd.Series(name='inchi_key', dtype=str)
for smi in smiles_to_add:
    mol = Chem.MolFromSmiles(smi)
    try:
        Chem.SanitizeMol(mol)
        ser.loc[len(ser)] = Chem.MolToInchiKey(mol)
    except Exception as e:
        print(smi, 'failed:', e)

mask = ~ser.isin(df.inchi_key)
if mask.any():    
    with pd.HDFStore('config/zinc_stock.hdf5', 'w') as store:
        df = pd.concat([df, ser[mask]])
        store.put('/table', df)
        print(mask.sum(), 'compounds added')

Loading zinc stock ... Done


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17422844 entries, 0 to 9
Data columns (total 1 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   inchi_key  object
dtypes: object(1)
memory usage: 265.9+ MB


In [53]:
import pandas as pd

with pd.HDFStore('config/zinc_stock.hdf5', 'r') as store:
    print(store.info())
    print(store.keys())

<class 'pandas.io.pytables.HDFStore'>
File path: config/zinc_stock.hdf5
/                 [invalid_HDFStore node: group ``/table`` does not have a child named ``description``]
/table            frame        (shape->[1,1])                                                          
['/', '/table']


In [55]:
with pd.HDFStore('config/zinc_stock_orig.hdf5', 'r') as store:
    df = store.get('/table')

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17422831 entries, 0 to 325908
Data columns (total 1 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   inchi_key  object
dtypes: object(1)
memory usage: 265.9+ MB


In [29]:
df.head()

,inchi_key
0,RYYVLZVUVIJVGH-UHFFFAOYSA-N
1,HZZVJAQRINQKSD-RQJHMYQMSA-N
2,RRTKVYSLIGQWCO-UHFFFAOYSA-N
3,YAPQBXQYLJRXSA-UHFFFAOYSA-N
4,SZPBAPFUXAADQV-UHFFFAOYSA-N


In [30]:
df.tail()

,inchi_key
325904,OKYGVTBDEYVSBV-KPPDAEKUSA-N
325905,OKYGVTBDEYVSBV-WHXUTIOJSA-N
325906,RIHTXYUOTJTDKX-UHFFFAOYSA-N
325907,WQOVAHPYNIWPHG-CYBMUJFWSA-N
325908,WQOVAHPYNIWPHG-ZDUSSCGKSA-N


In [31]:
df.index.max(), len(df)

(17396070, 17422831)

In [32]:
df.index[df.index.duplicated()]

Index([     0,      1,      2,      3,      4,      5,      6,      7,      8,
            9,
       ...
       325899, 325900, 325901, 325902, 325903, 325904, 325905, 325906, 325907,
       325908],
      dtype='int64', length=309919)

In [42]:
from rdkit import Chem

smiles = ['CO', 'CI', 'CBr', 'CCBr', 'CCI', 'CN', 'CCN', 'C=O', 'CC=O', 'NN']
ser = pd.Series(name='inchi_key')
for smi in smiles:
    mol = Chem.MolFromSmiles(smi)
    Chem.SanitizeMol(mol)
    ser.loc[len(ser)] = Chem.MolToInchiKey(mol)
    print("%5s %s" % (smi, ser.iloc[-1]))

   CO OKKJLVBELUTLKV-UHFFFAOYSA-N
   CI INQOMBQAUSQDDS-UHFFFAOYSA-N
  CBr GZUXJHMPEANEGY-UHFFFAOYSA-N
 CCBr RDHPKYGYEGBMSE-UHFFFAOYSA-N
  CCI HVTICUPFWKNHNG-UHFFFAOYSA-N
   CN BAVYZALUXZFZLV-UHFFFAOYSA-N
  CCN QUSNBJAOOMFDIB-UHFFFAOYSA-N
  C=O WSFSSNUMVMOOMR-UHFFFAOYSA-N
 CC=O IKHGUXGNUITLKF-UHFFFAOYSA-N
   NN OAKJQQAXSVQMHS-UHFFFAOYSA-N


In [44]:
mask = ser.isin(df.inchi_key)

In [45]:
ser[~mask]

0    OKKJLVBELUTLKV-UHFFFAOYSA-N
1    INQOMBQAUSQDDS-UHFFFAOYSA-N
2    GZUXJHMPEANEGY-UHFFFAOYSA-N
3    RDHPKYGYEGBMSE-UHFFFAOYSA-N
4    HVTICUPFWKNHNG-UHFFFAOYSA-N
5    BAVYZALUXZFZLV-UHFFFAOYSA-N
6    QUSNBJAOOMFDIB-UHFFFAOYSA-N
7    WSFSSNUMVMOOMR-UHFFFAOYSA-N
8    IKHGUXGNUITLKF-UHFFFAOYSA-N
9    OAKJQQAXSVQMHS-UHFFFAOYSA-N
Name: inchi_key, dtype: object

In [47]:
pd.concat([df, ser[~mask]], axis=0).shape

(17422841, 1)

In [48]:
Chem.MolToInchiKey(mol)

'OAKJQQAXSVQMHS-UHFFFAOYSA-N'

In [49]:
with pd.HDFStore('config/zinc_stock.hdf5', 'w') as store:
    store.put('/table', df)

In [52]:
import pandas as pd
from rdkit import Chem

smiles = ['CO', 'CI', 'CBr', 'CCBr', 'CCI', 'CN', 'CCN', 'C=O', 'CC=O', 'NN']

with pd.HDFStore('config/zinc_stock.hdf5', 'r') as store:
    df = store.get('/table')
    
ser = pd.Series(name='inchi_key')
for smi in smiles:
    mol = Chem.MolFromSmiles(smi)
    Chem.SanitizeMol(mol)
    ser.loc[len(ser)] = Chem.MolToInchiKey(mol)    
mask = ~(ser.isin(df.inchi_key))
df = pd.concat([df, ser[mask]], axis=0)

with pd.HDFStore('config/zinc_stock.hdf5', 'w') as store:
    store.put('/table', df)
    print(mask.sum(), 'compounds added')

10 compounds added
